# IPL EDA & Deep learning Transformers

## Step 1: Data Loading

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define the file path
balls_file_path = "/content/drive/MyDrive/IPL/ball_by_ball.csv"
match_file_path="/content/drive/MyDrive/IPL/matches.csv"

# Load the CSV file
ball_by_ball = pd.read_csv(balls_file_path)
matches = pd.read_csv(match_file_path)

In [5]:
ball_by_ball.head()

,ID,Innings,Overs,BallNumber,Batter,Bowler,NonStriker,ExtraType,BatsmanRun,ExtrasRun,TotalRun,NonBoundary,IsWicketDelivery,PlayerOut,Kind,FieldersInvolved,BattingTeam
0,1082591,1,0,1,DA Warner,TS Mills,S Dhawan,NaN,0,0,0,0,0,NaN,NaN,NaN,Sunrisers Hyderabad
1,1082591,1,0,2,DA Warner,TS Mills,S Dhawan,NaN,0,0,0,0,0,NaN,NaN,NaN,Sunrisers Hyderabad
2,1082591,1,0,3,DA Warner,TS Mills,S Dhawan,NaN,4,0,4,0,0,NaN,NaN,NaN,Sunrisers Hyderabad
3,1082591,1,0,4,DA Warner,TS Mills,S Dhawan,NaN,0,0,0,0,0,NaN,NaN,NaN,Sunrisers Hyderabad
4,1082591,1,0,5,DA Warner,TS Mills,S Dhawan,wides,0,2,2,0,0,NaN,NaN,NaN,Sunrisers Hyderabad


In [6]:
matches.head()

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1082591,Hyderabad,2017-04-05,2017,1,Sunrisers Hyderabad,Royal Challengers Bangalore,"Rajiv Gandhi International Stadium, Uppal",Royal Challengers Bangalore,field,N,Sunrisers Hyderabad,Runs,35,Yuvraj Singh,"['DA Warner', 'S Dhawan', 'MC Henriques', 'Yuv...","['CH Gayle', 'Mandeep Singh', 'TM Head', 'KM J...",AY Dandekar,NJ Llong
1,1082592,Pune,2017-04-06,2017,2,Rising Pune Supergiant,Mumbai Indians,Maharashtra Cricket Association Stadium,Rising Pune Supergiant,field,N,Rising Pune Supergiant,Wickets,7,SPD Smith,"['AM Rahane', 'MA Agarwal', 'SPD Smith', 'BA S...","['PA Patel', 'JC Buttler', 'RG Sharma', 'N Ran...",A Nand Kishore,S Ravi
2,1082593,Rajkot,2017-04-07,2017,3,Gujarat Lions,Kolkata Knight Riders,Saurashtra Cricket Association Stadium,Kolkata Knight Riders,field,N,Kolkata Knight Riders,Wickets,10,CA Lynn,"['JJ Roy', 'BB McCullum', 'SK Raina', 'AJ Finc...","['G Gambhir', 'CA Lynn', 'RV Uthappa', 'MK Pan...",Nitin Menon,CK Nandan
3,1082594,Indore,2017-04-08,2017,4,Kings XI Punjab,Rising Pune Supergiant,Holkar Cricket Stadium,Kings XI Punjab,field,N,Kings XI Punjab,Wickets,6,GJ Maxwell,"['HM Amla', 'M Vohra', 'WP Saha', 'AR Patel', ...","['AM Rahane', 'MA Agarwal', 'SPD Smith', 'BA S...",AK Chaudhary,C Shamshuddin
4,1082595,Bengaluru,2017-04-08,2017,5,Royal Challengers Bangalore,Delhi Daredevils,M.Chinnaswamy Stadium,Royal Challengers Bangalore,bat,N,Royal Challengers Bangalore,Runs,15,KM Jadhav,"['CH Gayle', 'SR Watson', 'Mandeep Singh', 'KM...","['AP Tare', 'SW Billings', 'KK Nair', 'SV Sams...",S Ravi,VK Sharma


## Step 2: Data Preprocessing

Sorting matches dataset and ball_by_ball dataset by season

In [8]:
matches = matches.sort_values(by=['Season', 'ID']).reset_index(drop=True)
ball_by_ball = ball_by_ball.sort_values(by=['ID', 'Innings', 'Overs', 'BallNumber']).reset_index(drop=True)

Handle Missing Values

In [12]:
print(matches.isnull().sum())
print(ball_by_ball.isnull().sum())

ID                 0
City               0
Date               0
Season             0
MatchNumber        0
Team1              0
Team2              0
Venue              0
TossWinner         0
TossDecision       0
SuperOver          0
WinningTeam        0
WonBy              0
Margin             0
Player_of_Match    5
Team1Players       0
Team2Players       0
Umpire1            0
Umpire2            0
dtype: int64
ID                       0
Innings                  0
Overs                    0
BallNumber               0
Batter                   0
Bowler                   0
NonStriker               0
ExtraType           246795
BatsmanRun               0
ExtrasRun                0
TotalRun                 0
NonBoundary              0
IsWicketDelivery         0
PlayerOut           247970
Kind                247970
FieldersInvolved    251566
BattingTeam              0
dtype: int64
